<a href="https://colab.research.google.com/github/macchino/macchino.github.io/blob/master/Tokyo_yakitori_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install MeCab
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install -U mecab-python3 > /dev/null
#!pip install unidic-lite
#!pip install ipadic

In [ ]:
# check path to "ipadic-neologd" 
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

In [ ]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import word2vec
import MeCab

tagger = MeCab.Tagger('-Owakati -r /dev/null -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]



#学習データの読み込み

df = pd.read_csv('tokyo_yaktori_review.csv')
df_yaktori = df.groupby(['store_name','score','review_cnt'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)

#コーパス作成
wakati_yaktori_text = []
for i in df_yaktori['review']:
    txt = tokenize(i, 2, 10000, True)
    wakati_yaktori_text.append(txt)
#↓↓↓↓↓↓↓ あなたの記事の内容
np.savetxt("yaktori_corpus.txt", wakati_yaktori_text,fmt='%s', delimiter=',')
#───────
np.savetxt("yaktori_corpus.txt", wakati_yaktori_text, fmt = '%s', delimiter = ',')
#↑↑↑↑↑↑↑ 編集リクエストの内容
# モデル作成
#↓↓↓↓↓↓↓ あなたの記事の内容
word2vec_yaktori_model = word2vec.Word2Vec(wakati_yaktori_text,sg=1,size=100, window=5,min_count=5,iter=100,workers=3)
#───────
word2vec_ramen_model = word2vec.Word2Vec(wakati_yaktori_text, sg = 1, size = 100, window = 5, min_count = 5, iter = 100, workers = 3)
#↑↑↑↑↑↑↑ 編集リクエストの内容
#sg（0: CBOW, 1: skip-gram）,size（ベクトルの次元数）,window（学習に使う前後の単語数）,min_count（n回未満登場する単語を破棄）,iter（トレーニング反復回数）

# モデルのセーブ
word2vec_yaktori_model.save("word2vec_yaktori_model.model")

In [ ]:
# モデルのロード
word2vec_yaktori_model =word2vec.Word2Vec.load("word2vec_yaktori_model.model")
word2vec_yaktori_model.most_similar("海鮮")

In [ ]:
#足し算
word2vec_yaktori_model.most_similar(positive=[u"銀座",u"焼鳥"])

In [ ]:
#引き算
word2vec_yaktori_model.most_similar(positive=[u"焼鳥",u"炭火"], negative=[u"銀座"])